In [1]:
import os
import fnmatch
from copy import copy
import itertools
from datetime import datetime
from pathlib import Path
from collections import namedtuple

import geowombat as gw
# from tsplanet import download_data

import geopandas as gpd
import numpy as np
import numba as nb
from skimage.exposure import rescale_intensity
import cv2
import sklearn_crfsuite
from affine import Affine
import matplotlib.pyplot as plt

In [2]:
import sklearn
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV

from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

## Functions

In [3]:
# str_to_int_dict = dict(n=0, a=1, g=1, t=1, u=1, b=1, w=2, c=3, s=4, h=5)
# str_to_int_dict = dict(n=0, a=1, g=2, t=3, u=4, b=5, w=6, c=7, s=8, h=9)

str_to_int_dict = dict(n=0, l=1, u=2, w=3, c=4, s=5, h=6)

int_to_str_dict = dict()

for k, v in str_to_int_dict.items():

    # Change clear land values to 'l'
#     if v == 1:
#         int_to_str_dict[v] = 'l'
#     else:
    int_to_str_dict[v] = k

In [4]:
str_to_int_dict

{'n': 0, 'l': 1, 'u': 2, 'w': 3, 'c': 4, 's': 5, 'h': 6}

In [5]:
def str_to_int(row):
    str_values = row['class'].split(',')
    return ','.join([str(str_to_int_dict[sv]) for sv in str_values])

In [6]:
def label_decoder(label):
    return int_to_str_dict[label]


def get_sample_xy(sample):
    
    """
    Converts a DataFrame sample to a CRF feature
    """
    
    date_diff = sample.date2.to_pydatetime().toordinal() - sample.date1.to_pydatetime().toordinal()
    
    nir1_diff = int(sample.nir1) - int(sample.nir1_1)
    nir2_diff = int(sample.nir2) - int(sample.nir2_1)
    nir3_diff = int(sample.nir3) - int(sample.nir3_1)
    rededge_diff = int(sample.rededge) - int(sample.rededge_1)
    swir1_diff = int(sample.swir1) - int(sample.swir1_1)
    swir2_diff = int(sample.swir2) - int(sample.swir2_1)
    
    nir1_diff_2 = int(sample.nir1_2) - int(sample.nir1)
    nir2_diff_2 = int(sample.nir2_2) - int(sample.nir2)
    nir3_diff_2 = int(sample.nir3_2) - int(sample.nir3)
    rededge_diff_2 = int(sample.rededge_2) - int(sample.rededge)
    swir1_diff_2 = int(sample.swir1_2) - int(sample.swir1)
    swir2_diff_2 = int(sample.swir2_2) - int(sample.swir2)
    
    diff1 = ['nir1-diff1', 'nir2-diff1', 'nir3-diff1', 'rededge-diff1', 'swir1-diff1', 'swir2-diff1']
    diff2 = ['nir1-diff2', 'nir2-diff2', 'nir3-diff2', 'rededge-diff2', 'swir1-diff2', 'swir2-diff2']
    
    slab = band_names + ['date-diff'] + diff1 + diff2
    
    diff1_values = [nir1_diff, nir2_diff, nir3_diff, rededge_diff, swir1_diff, swir2_diff]
    diff2_values = [nir1_diff_2, nir2_diff_2, nir3_diff_2, rededge_diff_2, swir1_diff_2, swir2_diff_2]
    
    sfea = np.array(sample[band_names].values.tolist() + [date_diff] + diff1_values + diff2_values, dtype='float64')

    return dict(zip(slab, sfea)), label_decoder(int(sample.poly))


def get_array_xy(sample):
    return dict(zip(band_names, sample))

    
def samples_to_features(dfs):
    
    """
    sample = [[{t1}, {t2}, ..., {tn}], ..., [...]]
    """
    
    return [list(zip(*df.apply(get_sample_xy, axis=1))) for df in dfs]


@nb.jit
def sample_to_dict(array_sample):

    diff1 = ['nir1-diff1', 'nir2-diff1', 'nir3-diff1', 'rededge-diff1', 'swir1-diff1', 'swir2-diff1']
    diff2 = ['nir1-diff2', 'nir2-diff2', 'nir3-diff2', 'rededge-diff2', 'swir1-diff2', 'swir2-diff2']
    
    slab = band_names + ['date-diff'] + diff1 + diff2    

    return dict(zip(slab, np.float64(array_sample)))    


@nb.jit
def array_to_dict(*args):
    
    """
    Converts array sample to a CRF features
    """
    
    return [sample_to_dict(sample) for sample in args]

In [7]:
def setup_names(grid, filenames):
    
    rpath_dict = {'t55jel': '/scratch/rsc8/hardtkel/biocondition/biocondition/layers/sentinel/{}'.format(grid),
                  't56jkt': '/scratch/rsc8/hardtkel/biocondition/biocondition/layers/sentinel/{}'.format(grid),
                  't55kcu': '/scratch/rsc8/hardtkel/rapidfires/LW/{}'.format(grid)}    
    
    NameInfo = namedtuple('NameInfo', 'rasters vector, time_names')
    
    vector_dir = Path('/scratch/rsc4/graesser/temp/s2/training')
    
    image_dir = Path(rpath_dict[grid])
    
    filename_dict = dict()
    for fn in filenames:
        filename_dict[fn.split('_')[2]] = fn

    sorted_names = list(zip(*sorted(filename_dict.items())))
    
    filenames = list(sorted_names[1])
    
    time_names = [datetime.strptime(t, '%Y%m%d') for t in list(sorted_names[0])]
    
    rasters = [image_dir.joinpath(fn).as_posix() for fn in filenames]
    vector = vector_dir.joinpath('{}.shp'.format(grid)).as_posix()
    
    return NameInfo(rasters=rasters, vector=vector, time_names=time_names)

## Extract samples from training AOIs

In [8]:
X = list()
y = list()

In [17]:
# grid = 't56jkt'
# grid = 't55jel'
grid = 't55kcu'

fnames_dict = {'t55jel': ['cemsre_t55jel_20171207_abbm5.img',
                          'cemsre_t55jel_20171227_abbm5.img',
                          'cemsre_t55jel_20180116_abbm5.img',
                          'cemsre_t55jel_20180317_abbm5.img',
                          'cemsre_t55jel_20180327_abbm5.img',
                          'cemsre_t55jel_20180406_abbm5.img',
                          'cemsre_t55jel_20180416_abbm5.img',
                          'cemsre_t55jel_20180426_abbm5.img',
                          'cemsre_t55jel_20180526_abbm5.img',
                          'cemsre_t55jel_20180605_abbm5.img',
                          'cemsre_t55jel_20180615_abbm5.img',
                          'cemsre_t55jel_20180715_abbm5.img',
                          'cemsre_t55jel_20180804_abbm5.img'],
              't56jkt': ['cemsre_t56jkt_20180423_abbm6.img',
                         'cemsre_t56jkt_20180513_abbm6.img',
                         'cemsre_t56jkt_20180523_abbm6.img',
                         'cemsre_t56jkt_20180602_abbm6.img'],
              't55kcu': ['cemsre_t55kcu_20171210_abbm5.img',
                         'cemsre_t55kcu_20171220_abbm5.img',
                         'cemsre_t55kcu_20171230_abbm5.img',
                         'cemsre_t55kcu_20180109_abbm5.img',
                         'cemsre_t55kcu_20180129_abbm5.img']}

rpath_dict = {'t55jel': '/scratch/rsc8/hardtkel/biocondition/biocondition/layers/sentinel/{}'.format(grid),
              't56jkt': '/scratch/rsc8/hardtkel/biocondition/biocondition/layers/sentinel/{}'.format(grid),
              't55kcu': '/scratch/rsc8/hardtkel/rapidfires/LW/{}'.format(grid)}

filenames = fnames_dict[grid]

band_names = ['nir1', 'nir2', 'nir3', 'rededge', 'swir1', 'swir2']
label_name = 'poly'

In [18]:
vector_dir = Path('/scratch/rsc4/graesser/temp/s2/training')
image_dir = Path(rpath_dict[grid])

filename_dict = dict()
for fn in filenames:
    filename_dict[fn.split('_')[2]] = fn
    
sorted_names = list(zip(*sorted(filename_dict.items())))

filenames = list(sorted_names[1])

time_names = [datetime.strptime(t, '%Y%m%d') for t in list(sorted_names[0])]

rasters = [image_dir.joinpath(fn).as_posix() for fn in filenames]
vector = vector_dir.joinpath('{}.shp'.format(grid)).as_posix()

In [19]:
df = gpd.read_file(vector)

df['class_len'] = df['class'].str.len()
df['int_class'] = df.apply(str_to_int, axis=1)
out = df['int_class'].str.split(',', expand=True)

df_list = list()

for s in range(0, out.shape[1]):
    df_ = df.copy()
    df_['id'] = out.iloc[:, s]
    df_list.append(df_)

print('Extracting samples ...')    
    
# Extract samples    
dfs = list()

with gw.config.update(sensor='s220'):

    with gw.open(rasters, time_names=time_names, chunks=512) as ds:    
       
        for ti, time in enumerate(ds.time.values):

            dss = ds.sel(time=time)
            
            dfe = dss.gw.extract(df_list[ti])
            
            if ti == 0:
                
                dfe['date1'] = time
                dfe['date2'] = time

                dfe['nir1_1'] = dfe.nir1
                dfe['nir2_1'] = dfe.nir2
                dfe['nir3_1'] = dfe.nir3
                dfe['rededge_1'] = dfe.rededge
                dfe['swir1_1'] = dfe.swir1
                dfe['swir2_1'] = dfe.swir2
                
            else:
                
                dfe['date1'] = ds.time.values[ti-1]
                dfe['date2'] = time
                
                dfe['nir1_1'] = dfs[-1].nir1
                dfe['nir2_1'] = dfs[-1].nir2
                dfe['nir3_1'] = dfs[-1].nir3
                dfe['rededge_1'] = dfs[-1].rededge
                dfe['swir1_1'] = dfs[-1].swir1
                dfe['swir2_1'] = dfs[-1].swir2
            
            dfs.append(dfe)    
            
        for ti, time in enumerate(ds.time.values):
            
            dfe = dfs[ti]

            if ti < len(ds.time.values)-1:
                
                dfe['nir1_2'] = dfs[ti+1].nir1
                dfe['nir2_2'] = dfs[ti+1].nir2
                dfe['nir3_2'] = dfs[ti+1].nir3
                dfe['rededge_2'] = dfs[ti+1].rededge
                dfe['swir1_2'] = dfs[ti+1].swir1
                dfe['swir2_2'] = dfs[ti+1].swir2                

            else:
                
                dfe['nir1_2'] = dfe.nir1
                dfe['nir2_2'] = dfe.nir2
                dfe['nir3_2'] = dfe.nir3
                dfe['rededge_2'] = dfe.rededge
                dfe['swir1_2'] = dfe.swir1
                dfe['swir2_2'] = dfe.swir2
                
            dfs[ti] = dfe
            
print('Converting samples to features ...')            
            
# Convert samples to features            
results = samples_to_features(dfs)
Xt = np.array(list(zip(*results))[0]).T.tolist()
yt = np.array(list(zip(*results))[1]).T.tolist()            

print('Finished')

Extracting samples ...


100%|██████████| 25/25 [00:00<00:00, 157.11it/s]


Converting samples to features ...
Finished


In [20]:
X += Xt
y += yt

### Fit the CRF model

In [21]:
# Gradient descent using the limited-memory BFGS method (with L1 and L2 regularization)
model = sklearn_crfsuite.CRF(algorithm='lbfgs', 
                             c1=0.001,
                             c2=0.001,
                             max_iterations=5000,
                             num_memories=10,
                             epsilon=0.0001,
                             delta=0.0001,
                             period=10,
                             linesearch='StrongBacktracking',  # 'MoreThuente' 'Backtracking' 'StrongBacktracking'
                             max_linesearch=20,
                             all_possible_states=True,
                             all_possible_transitions=True,
                             verbose=False)

In [22]:
# params_space = {'c1': [0.1, 0.01],
#                 'c2': [0.1, 0.01],
#                 'epsilon': [0.1, 0.01],
#                 'delta': [0.1, 0.01]}

# # use the same metric for evaluation
# f1_scorer = make_scorer(metrics.flat_f1_score,
#                         average='weighted', 
#                         labels=y)

# # search
# rs = RandomizedSearchCV(model, 
#                         params_space,
#                         cv=2,
#                         verbose=2,
#                         n_jobs=16,
#                         n_iter=20,
#                         scoring=f1_scorer)

# rs.fit(X, y)

In [23]:
# print('best params:', rs.best_params_)
# print('best CV score:', rs.best_score_)
# print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

### Open the data to predict on 

In [25]:
image_info = setup_names('t55kcu', fnames_dict['t55kcu'])

In [26]:
with gw.config.update(sensor='s2l7'):
    
    with gw.open(image_info.rasters, 
                 time_names=image_info.time_names, 
                 chunks=512) as ds:
        
        dss = gw.subset(ds, left=348713.414, top=7820035.516, rows=512+256, cols=512+256)
        
        test = dss.astype('float64').data.compute()

In [27]:
test.shape

(5, 6, 768, 768)

In [28]:
ntime, nbands, nrows, ncols = test.shape

### Add date ordinals

In [29]:
stacked_arrays = list()

for di in range(0, len(image_info.time_names)):
    
    ordinal_array = np.zeros((nrows, ncols), dtype='uint64')
    
    nir1_diff_array1 = np.zeros((nrows, ncols), dtype='uint64')
    nir2_diff_array1 = nir1_diff_array1.copy()
    nir3_diff_array1 = nir1_diff_array1.copy()
    rededge_diff_array1 = nir1_diff_array1.copy()
    swir1_diff_array1 = nir1_diff_array1.copy()
    swir2_diff_array1 = nir1_diff_array1.copy()
    
    nir1_diff_array2 = nir1_diff_array1.copy()
    nir2_diff_array2 = nir1_diff_array1.copy()
    nir3_diff_array2 = nir1_diff_array1.copy()
    rededge_diff_array2 = nir1_diff_array1.copy()
    swir1_diff_array2 = nir1_diff_array1.copy()
    swir2_diff_array2 = nir1_diff_array1.copy()    
    
    if di > 0:
        
        date_diff = image_info.time_names[di].toordinal() - image_info.time_names[di-1].toordinal()

        #date_diff = dfs[di].iloc[0].date2.to_pydatetime().toordinal() - dfs[di-1].iloc[0].date1.to_pydatetime().toordinal()
        ordinal_array = ordinal_array + date_diff
        
        nir1_diff_array1 = test[di][0] - test[di-1][0]
        nir2_diff_array1 = test[di][1] - test[di-1][1]
        nir3_diff_array1 = test[di][2] - test[di-1][2]
        rededge_diff_array1 = test[di][3] - test[di-1][3]
        swir1_diff_array1 = test[di][4] - test[di-1][4]
        swir2_diff_array1 = test[di][5] - test[di-1][5]
        
    if di < len(image_info.time_names)-1:
        
        nir1_diff_array2 = test[di+1][0] - test[di][0]
        nir2_diff_array2 = test[di+1][1] - test[di][1]
        nir3_diff_array2 = test[di+1][2] - test[di][2]
        rededge_diff_array2 = test[di+1][3] - test[di][3]
        swir1_diff_array2 = test[di+1][4] - test[di][4]
        swir2_diff_array2 = test[di+1][5] - test[di][5]
    
    stacked_arrays.append(np.concatenate((test[di], 
                                          ordinal_array[np.newaxis, :, :],
                                          nir1_diff_array1[np.newaxis, :, :],
                                          nir2_diff_array1[np.newaxis, :, :],
                                          nir3_diff_array1[np.newaxis, :, :],
                                          rededge_diff_array1[np.newaxis, :, :],
                                          swir1_diff_array1[np.newaxis, :, :],
                                          swir2_diff_array1[np.newaxis, :, :],
                                          nir1_diff_array2[np.newaxis, :, :],
                                          nir2_diff_array2[np.newaxis, :, :],
                                          nir3_diff_array2[np.newaxis, :, :],
                                          rededge_diff_array2[np.newaxis, :, :],
                                          swir1_diff_array2[np.newaxis, :, :],
                                          swir2_diff_array2[np.newaxis, :, :])))
    
stacked_arrays = np.array(stacked_arrays)    

In [30]:
stacked_arrays.shape

(5, 8, 768, 768)

In [31]:
ntime, nbands, nrows, ncols = stacked_arrays.shape

### Convert the array to CRF-compatible features

In [32]:
features = [tlayer.transpose(1, 2, 0).reshape(nrows*ncols, nbands) for tlayer in stacked_arrays]
features = [array_to_dict(*[features[j][i] for j in range(0, ntime)]) for i in range(0, nrows*ncols)]

In [33]:
# from geowombat.models import time_to_crffeas
# import numpy as np
# ntime,nbands,nrows,ncols=3,6,5,5
# stacked_arrays = np.ascontiguousarray(np.arange(ntime*nbands*nrows*ncols).reshape(ntime,nbands,nrows,ncols), dtype='float64')

# features = dict()
# for ti in range(0, stacked_arrays.shape[0]):
#     features[ti] = np.ascontiguousarray(stacked_arrays[ti].transpose(1, 2, 0).reshape(nrows*ncols, nbands), dtype='float64')

# time_to_crffeas(features, list(range(6)), 6)

### Predict on all time steps

In [34]:
pred = model.predict_marginals(features)

In [35]:
def pred_to_labels(model_pred):
    
    # samples x time x n classes
    return np.array([[[ps['l'], ps['w'], ps['c'], ps['s'], ps['h']] for ps in p] for p in model_pred], dtype='float64')

In [36]:
# samples x time x n classes
# pred = np.array([[[ps['n'], ps['l'], ps['w'], ps['c'], ps['s'], ps['h']] for ps in p] 
#                  for p in model.predict_marginals(features)], dtype='float64')

In [37]:
pred = pred_to_labels(pred)

In [38]:
pred.shape

(589824, 5, 5)

In [39]:
pred = pred.transpose(1, 2, 0).reshape(ntime, pred.shape[2], nrows, ncols)

In [40]:
pred.shape

(5, 5, 768, 768)

## Plot probabilities

In [ ]:
%matplotlib notebook

fig, (ax1, ax2) = plt.subplots(1, 2, dpi=120)

# pr_max = pred[: 2, :, :].max(axis=0)
idx = 3

#proba = pred[idx, 2:, :, :].max(axis=0) #- pred[:idx-1, 2, :, :].cumsum(axis=0)[-1, :, :]
pred_int = pred[idx, :, :, :].argmax(axis=0)
pred_int = np.uint8(np.where(pred_int == 2, 1, np.where(pred_int == 3, 2, 0)))
pred_int = cv2.dilate(pred_int, np.array([[0, 1, 0], [1, 1, 1], [0, 1, 0]], dtype='uint8'), iterations=3)

ip = test[idx, 2:5, :, :]
ip = rescale_intensity(ip, 
                       in_range=(ip.min(), ip.max()), 
                       out_range=(np.nanpercentile(ip, 10), 
                                  np.nanpercentile(ip, 95)))
ip = np.flip(ip, axis=0)
ip = np.ascontiguousarray(ip.transpose(1, 2, 0))
ip /= ip.max()

ax1.imshow(ip)
#ax1.imshow(ip, vmin=np.nanpercentile(ip, 5), vmax=np.nanpercentile(ip, 95), cmap='viridis')

#ax2.imshow(proba, vmin=np.nanpercentile(proba, 5), vmax=np.nanpercentile(proba, 95), cmap='magma')
ax2.imshow(pred_int, vmin=0, vmax=2)

ax1.axis('off')
ax2.axis('off')

plt.tight_layout(pad=0.1)

plt.show()

In [ ]:
pred.shape

In [ ]:
fig, axes = plt.subplots(pred.shape[0], 2, dpi=400, figsize=(2, 12))

bidx = 0 # band index
cidx = 2 # class index

for pi in range(0, pred.shape[0]):

    ip = test[pi, bidx, :, :]
    #ip[ip == 0] = np.nan
    
#     pr = pred[pi, cidx, :, :]
#     pr = (pr_max - pr) / (pr_max + pr)
#     pr = (pr + pr_max) / 2.0
    
    pred_int = pred[pi, :, :, :].argmax(axis=0)
    pred_int = np.where(pred_int == 2, 1, np.where(pred_int == 3, 2, 0))
    
    axes[pi][0].imshow(ip, vmin=np.nanpercentile(ip, 5), vmax=np.nanpercentile(ip, 95), cmap='viridis')
    #axes[pi][1].imshow(pr, vmin=np.nanpercentile(pr, 5), vmax=np.nanpercentile(pr, 95), cmap='bone')
    axes[pi][1].imshow(pred_int, vmin=0, vmax=2)
    
    axes[pi][0].axis('off')
    axes[pi][1].axis('off')

plt.show()